In [325]:
import numpy
import pandas as pd
import requests
import math
from bs4 import BeautifulSoup
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer

In [326]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [327]:
URL1 = "https://id.theasianparent.com/11-resep-masakan-indonesia-populer"
URL2 = "https://www.masakapahariini.com/resep-masakan/masakan-tradisional/"
URL3 = "https://lestariweb.com/AyamGeprek.php"
r1 = requests.get(URL1)
r2 = requests.get(URL2)
r3 = requests.get(URL3)
soup1 = BeautifulSoup(r1.content, 'html5lib')
soup2 = BeautifulSoup(r2.content, 'html5lib')
soup3 = BeautifulSoup(r3.content, 'html5lib')
# soup.prettify()

In [328]:
docList = [soup1,soup2,soup3]

In [329]:
def createDocument(document):
    documents = document.find_all('div')
    for content in documents:
        content = content.get_text()
        return content

In [330]:
def creatListDocument(listDoc):
    docs = []
    for i in docList:
        docs.append(createDocument(i))
        
    return docs

In [331]:
# creatListDocument(docList)

In [332]:
def listOfWord(listDocument):
    listOfWords = []
    
    for sentence in listDocument:
        for word in stemmer.stem(sentence).split(" "):
            if word not in listOfWords:
                listOfWords.append(word)
                
    return listOfWords

In [333]:
def createNumOfWords(listOfWord,lenDocument):
    TF = []
    
    for i in range(lenDocument):
        TF.append(dict(zip(listOfWord, [0 for i in range(len(listOfWord))])))
    
    return TF

In [334]:
def computeTF(numOfWords, listDocument):
    tfDict = []
    bagWords = []
    for sentence in listDocument:
        bagWords.append(stemmer.stem(sentence).split(" "))
    
    for i in range(len(numOfWords)):
        dictTF = {}
        for key, value in numOfWords[i].items():
            dictTF[key] = value / float(len(bagWords[i]))
        tfDict.append(dictTF)
#         dictTF = {}
    return tfDict

In [349]:
def computeIDF(numOfWords):
    N = len(numOfWords)
    
    idfDict = dict.fromkeys(numOfWords[0].keys(), 0)
    for doc in numOfWords:
        for key, value in doc.items():
            if value > 0:
                idfDict[key] += 1
#     print (idfDict)
    for key, value in idfDict.items():
        try:
            idfDict[key] = round(math.log(N/float(value)),3)
        except ZeroDivisionError:
            idfDict[key] = 0
        
    
    return idfDict

In [380]:
def computeTFIDF(computeTF,idf):
    tfidfDict = []
    for i in range(len(computeTF)):
        dictTFIDF = {}
        for key, value in computeTF[i].items():
            dictTFIDF[key] = value * idf[key]
        tfidfDict.append(dictTFIDF)
    return tfidfDict

In [381]:
listOfWord = listOfWord(creatListDocument(docList))

TypeError: 'list' object is not callable

In [382]:
listOfWord

['ramadan',
 '2021gizi',
 'stimulasihidrasi',
 'keluargacek',
 'alergisukses',
 'asi',
 'eksklusifcari',
 'nama',
 'bayikehamilanproject',
 'sidekickstips',
 'kehamilantrimester',
 'pertamatrimester',
 'keduatrimester',
 'ketigamelahirkanmenyusuikehilangan',
 'bayitumbuh',
 'kembangbayibalitaprasekolahusia',
 'sekolahpraremajaparentingkeluargapernikahanseksberita',
 'terkinikesehatancovid-19info',
 'sehatpenyakitvaksinasikebugarangaya',
 'hiduphiburantravelfashionkecantikankeuanganmarvelous',
 'mums',
 'special',
 '2021nutrisiresepmakanan',
 'minumanvideoskata',
 'pakar',
 'parentingpilihan',
 'parentsplesiran',
 'ramah',
 'anakkisah',
 'keluargaeventkesehatankehamilantumbuh',
 'kembangbelanjatravel11',
 'resep',
 'masakan',
 'indonesia',
 'populerbacaan',
 '11',
 'menit',
 'bingung',
 'susun',
 'menu',
 'hari',
 'ini',
 'simak',
 'populer',
 'pilih',
 'kami',
 'mungkin',
 'bisa',
 'jadi',
 'inspirasi',
 'ingin',
 'coba',
 'bagai',
 'sangat',
 'agam',
 'seringkali',
 'sama',
 'tetapi',

In [383]:
lenListDocuments = len(docList)

In [384]:
numOfWords = createNumOfWords(listOfWord,lenListDocuments)

In [385]:
listDocs = []
for i in creatListDocument(docList):
    listDocs.append(i)
for index, sentence in enumerate(listDocs):
#     print(index,sentence)
    for word in stemmer.stem(sentence).split(" "):
        if word in TF[index]:
            numOfWords[index][word]+=1

In [386]:
len(numOfWords)

3

In [387]:
# numOfWords

In [388]:
computeTF = computeTF(numOfWords, creatListDocument(docList))

TypeError: 'list' object is not callable

In [389]:
# computeTF

In [390]:
computeIDF = computeIDF(numOfWords)

TypeError: 'dict' object is not callable

In [391]:
computeTFIDF = computeTFIDF(computeTF,computeIDF)

In [392]:
df = pd.DataFrame(computeTFIDF)

In [393]:
df

,ramadan,2021gizi,stimulasihidrasi,keluargacek,alergisukses,asi,eksklusifcari,nama,bayikehamilanproject,sidekickstips,...,lestari,contact,us,sitemap,impressum,disclaimer,2009-,reserved,design,by
0,0.000461,0.000922,0.000922,0.000922,0.000922,0.001383,0.000922,0.000510,0.000461,0.000461,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000222,0.000000,0.000000,...,0.000604,0.000604,0.000604,0.000604,0.000604,0.000604,0.000604,0.000604,0.000604,0.000604


In [394]:
computeTFIDF

[{'ramadan': 0.00046098993288590603,
  '2021gizi': 0.0009219798657718121,
  'stimulasihidrasi': 0.0009219798657718121,
  'keluargacek': 0.0009219798657718121,
  'alergisukses': 0.0009219798657718121,
  'asi': 0.001382969798657718,
  'eksklusifcari': 0.0009219798657718121,
  'nama': 0.0005096476510067114,
  'bayikehamilanproject': 0.00046098993288590603,
  'sidekickstips': 0.00046098993288590603,
  'kehamilantrimester': 0.0009219798657718121,
  'pertamatrimester': 0.0009219798657718121,
  'keduatrimester': 0.0009219798657718121,
  'ketigamelahirkanmenyusuikehilangan': 0.00046098993288590603,
  'bayitumbuh': 0.00046098993288590603,
  'kembangbayibalitaprasekolahusia': 0.00046098993288590603,
  'sekolahpraremajaparentingkeluargapernikahanseksberita': 0.00046098993288590603,
  'terkinikesehatancovid-19info': 0.00046098993288590603,
  'sehatpenyakitvaksinasikebugarangaya': 0.00046098993288590603,
  'hiduphiburantravelfashionkecantikankeuanganmarvelous': 0.00046098993288590603,
  'mums': 0.0